<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg"  width=300, align="right">
<br>
<br>
<br>
<br>
<br>

# ***Título do seu projeto***
---

*Siga esse template para entrega do seu projeto. O código deste notebook precisa ser 100% executável.*

In [ ]:
#@title Identificação do Grupo

#@markdown Turma **7N**

#@markdown Integrantes do Grupo (*informe \<TIA\>,\<nome\>*)
Aluno1 = '31928900, Ítalo Assunção' #@param {type:"string"} 
Aluno2 = '31986773, Rafael Alencar' #@param {type:"string"}
Aluno3 = '31617212, Felipe Retondar' #@param {type:"string"}
Aluno4 = '31946968, Roberto Tsuneki' #@param {type:"string"}


# Apresentação do Trabalho

*Apresente resumidamente o trabalho justificando a escolha dos dados.*

# Referencial Teórico

*Resuma aqui referências sobre o uso da regressão linear $times$ regressão logística na classificação de dados.* 

# Base de Dados 

*Descreva aqui os dados utilizados, discuta eventuais transformações e/ou seleções dos dados e preparações nos dados.* 





In [ ]:
# seu código

# Modelo de Regressão Linear

*Descreva aqui as variáveis dependentes e independentes empregadas e eventuais transformações nas variáveis.*


In [ ]:
# seu código

## Análise do Modelo Linear

In [ ]:
# seu código

## Discussão dos Resultados do Modelo Linear

*Discuta aqui os resultados obtidos no modelo.*


# Modelo Logístico

*Descreva aqui as variáveis dependentes e independentes empregadas e eventuais transformações nas variáveis.*


In [ ]:
# seu código

## Análise do Modelo Logístico

In [ ]:
# seu código

## Discussão dos Resultados do Modelo Logístico

*Discuta aqui os resultados obtidos no modelo.*


In [ ]:
# seu código

# Conclusão 

*Apresente a conclusão do seu estudo comparando ainda os resultados obtidos com o referencial teórico apresentado.*



# Referências 

*Indique as referências empregadas, incluindo as fontes de dados.*

---

In [ ]:
#@title Avaliação
Completo = 10 #@param {type:"slider", min:0, max:10, step:1}
#@markdown Projeto cumpre todos os itens pedidos. 
Relevancia = 9 #@param {type:"slider", min:0, max:10, step:1}
#@markdown As seleções de dados e eventos para análise são relevantes e justificados. 
Tecnicas = 6 #@param {type:"slider", min:0, max:10, step:1}
#@markdown As técnicas de empregadas são adequadas e corretamente aplicadas.
Apresentacao = 7 #@param {type:"slider", min:0, max:10, step:1}
#@markdown A apresentação dos resultados é clara e objetiva.
Analise = 8 #@param {type:"slider", min:0, max:10, step:1}
#@markdown As premissas de análise se justificam e a analise é correta. 
Conclusao = 7 #@param {type:"slider", min:0, max:10, step:1}
#@markdown As conclusões são justificadas e relevantes
Bonus = 0.5 #@param {type:"slider", min:0, max:1, step:0.5}
#@markdown A critério do professor por inovações na abordagem e no uso de técnicas de Análise de Dados








In [ ]:
#@markdown ### Nota Final
nota = Completo + Relevancia + Tecnicas + Apresentacao + Analise + Conclusao 

nota = nota / 6 + Bonus

print(f'Nota final do trabalho {nota :.1f}')

import numpy as np
import pandas as pd

alunos = pd.DataFrame()

lista_tia = []
lista_nome = []

for i in range(1,6):
  exec("if Aluno" + str(i) + " !='None':  lista = Aluno" + str(i) + ".split(','); lista_tia.append(lista[0]); lista_nome.append(lista[1].upper())")

alunos['tia'] = lista_tia
alunos['nome'] = lista_nome
alunos['nota'] = np.round(nota,1)
print()
alunos

Nota final do trabalho 8.3



,tia,nome,nota
0,1115665,ADRIANA FUJITA,8.3
1,1115677,DANIEL HENRIQUE,8.3
